# Component guide

The notebook will describe the different component that can be added to the beamline,
their parameters, and how to inspect the neutrons that reach each component.

In [ ]:
import numpy as np
import scipp as sc
import tof

meter = sc.Unit('m')
Hz = sc.Unit('Hz')
deg = sc.Unit('deg')

We begin by making a source pulse using the profile from ESS.

In [ ]:
source = tof.Source(facility='ess', neutrons=1_000_000)
source.plot()

## Adding a detector

We first add a `Detector` component which simply records all the neutrons that reach it.
It does not block any neutrons, they all travel through the detector without being absorbed.

In [ ]:
detector = tof.Detector(distance=30.0 * meter, name='detector')

# Build the instrument model
model = tof.Model(source=source, detectors=[detector])
model

In [ ]:
# Run and plot the rays
res = model.run()
res.plot()

As expected, the detector sees all the neutrons from the pulse.
Each component in the instrument has a `.plot()` method,
which allows us to quickly visualize histograms of the neutron counts at the detector.

In [ ]:
res.detectors['detector'].plot()

The data itself is available via the `.toas`, `.wavelengths`, `.birth_times`, and `.speeds` properties,
depending on which one you wish to inspect.

Note that we here need to have the additional `.visible` property in the chain,
because components have both `.visible` and `.blocked` neutrons,
but only choppers have blocked data (it is `None` for detectors).

In [ ]:
res.detectors['detector'].toas.visible.data

## Adding a chopper

Next, we add a chopper in the beamline,
with a frequency, phase, distance from source,
and a set of open and close angles for the cutouts in the rotating disk.

In [ ]:
chopper1 = tof.Chopper(
    frequency=10.0 * Hz,
    open=sc.array(
        dims=['cutout'],
        values=[30.0, 50.0],
        unit='deg',
    ),
    close=sc.array(
        dims=['cutout'],
        values=[40.0, 80.0],
        unit='deg',
    ),
    phase=0.0 * deg,
    distance=8 * meter,
    name="Chopper1",
)
chopper1

We can directly set this on our existing model, and re-run the simulation.

In [ ]:
model.add(chopper1)
res = model.run()
res.plot()

As expected, the two openings now create two bursts of neutrons,
separating the wavelengths into two groups.

If we plot the chopper data,

In [ ]:
res.choppers['Chopper1'].toas.plot()

we notice that the chopper sees all the incoming neutrons,
and blocks many of them (gray), only allowing a subset to pass through the openings (blue).

The detector now sees two peaks in its histogrammed counts:

In [ ]:
res.detectors['detector'].toas.plot()

## Multiple choppers

It is of course possible to add more than one chopper.
Here we add a second one, further down the beam path,
which splits each of the groups into two more groups.

In [ ]:
chopper2 = tof.Chopper(
    frequency=5.0 * Hz,
    open=sc.array(
        dims=['cutout'],
        values=[30.0, 40.0, 55.0, 70.0],
        unit='deg',
    ),
    close=sc.array(
        dims=['cutout'],
        values=[35.0, 48.0, 65.0, 90.0],
        unit='deg',
    ),
    phase=0.0 * deg,
    distance=20 * meter,
    name="Chopper2",
)

model.add(chopper2)
res = model.run()
res.plot()

The distribution of neutrons that are blocked and pass through the second chopper looks as follows:

In [ ]:
res.choppers['Chopper2'].plot()

while the detector now sees 4 peaks

In [ ]:
res.detectors['detector'].plot()

To view the blocked rays on the time-distance diagram of the model, use

In [ ]:
res.plot(max_rays=100, blocked_rays=5000)

## Adding a monitor

Detectors can be placed anywhere in the beam path,
and in the next example we place a detector (which will act as a monitor) between the first and second chopper.

In [ ]:
monitor = tof.Detector(distance=15.0 * meter, name='monitor')

model.add(monitor)
res = model.run()
res.plot()

In [ ]:
res.detectors['monitor'].plot()

## Counter-rotating chopper

By default, choppers are rotating clockwise.
This means than when open and close angles of the chopper windows are defined as increasing angles in the anti-clockwise direction,
the first window (with the lowest opening angles) will be the first one to pass in front of the beam.

To make a chopper rotate in the anti-clockwise direction, use the `direction` argument:

In [ ]:
chopper = tof.Chopper(
    frequency=10.0 * Hz,
    open=sc.array(
        dims=['cutout'],
        values=[280.0, 320.0],
        unit='deg',
    ),
    close=sc.array(
        dims=['cutout'],
        values=[310.0, 330.0],
        unit='deg',
    ),
    direction=tof.AntiClockwise,
    phase=0.0 * deg,
    distance=8 * meter,
    name="Counter-rotating chopper",
)

model = tof.Model(source=source, detectors=[detector], choppers=[chopper])
res = model.run()

In [ ]:
res.plot()